In [1]:
from mpi4py import MPI
import numpy as np
import math

# Function to perform sequential quicksort
def sequential_quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return sequential_quicksort(left) + middle + sequential_quicksort(right)

# Function to perform parallel quicksort recursively
def parallel_quicksort_recursive(arr, pr_rank, max_rank, rank_index, size_pool):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    share_pr = pr_rank + math.pow(2, rank_index)
    rank_index += 1
    
    if share_pr > max_rank:
        # If no process to share, sort sequentially
        sorted_data = sequential_quicksort(arr)
        return sorted_data
    
    pivot = arr[len(arr)//2]
    partition_pt = sequential_quicksort(arr, pivot, len(arr), (len(arr)//2) - 1)
    
    offset = partition_pt + 1
    if offset > len(arr) - offset:
        send_data = arr[offset:]
        comm.Send(send_data, dest=share_pr, tag=offset)
        sorted_data = parallel_quicksort_recursive(arr[:offset], pr_rank, max_rank, rank_index, size_pool)
        recv_data = np.empty(len(arr) - offset, dtype=int)
        comm.Recv(recv_data, source=share_pr, tag=MPI.ANY_TAG)
        sorted_data += recv_data
    else:
        send_data = arr[:offset]
        comm.Send(send_data, dest=share_pr, tag=0)
        sorted_data = parallel_quicksort_recursive(arr[offset:], pr_rank, max_rank, rank_index, size_pool)
        recv_data = np.empty(offset, dtype=int)
        comm.Recv(recv_data, source=share_pr, tag=MPI.ANY_TAG)
        sorted_data = recv_data + sorted_data
    
    return sorted_data

# Main function
if __name__ == "__main__":
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Your main code here
    data_size = 100
    data = None
    
    if rank == 0:
        data = np.random.randint(0, 100, size=data_size)

    data = comm.bcast(data, root=0)
    
    sorted_data = parallel_quicksort_recursive(data, 0, size-1, 0, size)

    print(f"Rank {rank} sorted data: {sorted_data}")

    MPI.Finalize()


Rank 0 sorted data: [0, 1, 2, 3, 3, 4, 6, 6, 7, 8, 10, 12, 13, 14, 16, 18, 18, 19, 20, 20, 21, 22, 22, 22, 23, 25, 27, 29, 29, 30, 31, 32, 33, 34, 35, 35, 36, 38, 38, 39, 42, 42, 43, 44, 45, 45, 47, 50, 50, 50, 51, 52, 52, 52, 53, 54, 54, 55, 55, 57, 58, 59, 60, 61, 61, 62, 63, 65, 67, 67, 67, 70, 71, 72, 72, 72, 74, 75, 75, 77, 77, 78, 78, 79, 79, 81, 81, 85, 86, 86, 91, 91, 91, 92, 92, 93, 95, 97, 97, 99]
